In [85]:
import pandas as pd 
import sqlite3

conn = sqlite3.connect("data/chinook.db")

In [86]:
songs = pd.read_sql_query('''SELECT t.name as Song, g.name as Genre, a.title as Album,
ar.name as ArtistName, m.name as MediaTypes, t.unitprice as UnitPrice, t.Composer,
i.CustomerId, c.firstname|| ' ' ||c.lastname as customername, ii.Quantity,
i.InvoiceDate,i.Billingcity as City,i.billingCountry as Country, t.trackid
from tracks t
left join playlist_track on playlist_track.trackid = t.name
left join playlists p on playlist_track.playlistid = p.playlistid
left join media_types m on m.mediatypeid = t.mediatypeid
left join genres g on g.genreid = t.genreid
left join albums a on a.albumid = t.albumid
left join artists ar on ar.artistid = a.artistid
left join invoice_items ii on ii.trackid = t.trackid
left join invoices i on ii.invoiceid = i.invoiceid
left join customers c on i.customerid = c.customerid''', conn, parse_dates='InvoiceDate')

In [87]:
#Change data types
songs.dtypes

Song                    object
Genre                   object
Album                   object
ArtistName              object
MediaTypes              object
UnitPrice              float64
Composer                object
CustomerId             float64
customername            object
Quantity               float64
InvoiceDate     datetime64[ns]
City                    object
Country                 object
TrackId                  int64
dtype: object

In [88]:
songs[['Genre', 'ArtistName', 'MediaTypes', 'City', 'Country', 'Album']] = songs[['Genre', 'ArtistName', 'MediaTypes', 'City', 'Country', 'Album']].astype('category')

In [89]:
songs.head()

,Song,Genre,Album,ArtistName,MediaTypes,UnitPrice,Composer,CustomerId,customername,Quantity,InvoiceDate,City,Country,TrackId
0,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,MPEG audio file,0.99,"Angus Young, Malcolm Young, Brian Johnson",47.0,Lucas Mancini,1.0,2010-04-13,Rome,Italy,1
1,Balls to the Wall,Rock,Balls to the Wall,Accept,Protected AAC audio file,0.99,None,2.0,Leonie Köhler,1.0,2009-01-01,Stuttgart,Germany,2
2,Balls to the Wall,Rock,Balls to the Wall,Accept,Protected AAC audio file,0.99,None,33.0,Ellie Sullivan,1.0,2011-07-25,Yellowknife,Canada,2
3,Fast As a Shark,Rock,Restless and Wild,Accept,Protected AAC audio file,0.99,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",13.0,Fernanda Ramos,1.0,2012-11-01,Brasília,Brazil,3
4,Restless and Wild,Rock,Restless and Wild,Accept,Protected AAC audio file,0.99,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",2.0,Leonie Köhler,1.0,2009-01-01,Stuttgart,Germany,4


In [90]:
songs.dtypes

Song                    object
Genre                 category
Album                 category
ArtistName            category
MediaTypes            category
UnitPrice              float64
Composer                object
CustomerId             float64
customername            object
Quantity               float64
InvoiceDate     datetime64[ns]
City                  category
Country               category
TrackId                  int64
dtype: object

In [91]:
songs.shape

(3761, 14)

In [92]:
songs["Composer"].fillna('Unknown',inplace=True)
songs["Quantity"].fillna(0,inplace=True)

In [93]:
songs.isna().sum()

Song               0
Genre              0
Album              0
ArtistName         0
MediaTypes         0
UnitPrice          0
Composer           0
CustomerId      1521
customername    1521
Quantity           0
InvoiceDate     1521
City            1521
Country         1521
TrackId            0
dtype: int64

In [94]:
songs = songs.drop_duplicates()

In [95]:
songs = songs.dropna(how = 'any')

In [96]:
songs = songs.reset_index(drop = True)

In [97]:
#Top 10 songs most bought by customers
toptensongs = pd.crosstab(index = [songs.Song, songs.ArtistName],
                  columns = 'freq',
                  values = songs.Quantity,
                  aggfunc = 'sum').sort_values('freq', ascending = False)

In [98]:
top10songs = toptensongs.reset_index(level=['Song', 'ArtistName']).head(10)

In [99]:
top10songs

col_0,Song,ArtistName,freq
0,The Trooper,Iron Maiden,5.0
1,The Number Of The Beast,Iron Maiden,4.0
2,Hallowed Be Thy Name,Iron Maiden,4.0
3,Sure Know Something,Kiss,4.0
4,Eruption,Van Halen,4.0
5,Plaster Caster,Kiss,3.0
6,2 Minutes To Midnight,Iron Maiden,3.0
7,Heaven Can Wait,Iron Maiden,3.0
8,Can I Play With Madness,Iron Maiden,3.0
9,Release,The Tea Party,3.0


The reason why I did not just group it by track ID is because multiple of the same songs (but in different albums, aka repackaged albums) are considered different songs.
That is why I put the name of the artist there, so if the name of the song and the artist name matches I will count it as one song. (instead of albums because some songs are the same but are in different albums).
Above is the top 10 songs that are most bought by customers in this database.

In [100]:
#Top 10 Artists
toptenartists = toptensongs.stack().to_frame().groupby(['ArtistName']).sum().sort_values(0, ascending = False).head(10)

In [101]:
top10artists = toptenartists.reset_index(level='ArtistName')

In [102]:
top10artists

,ArtistName,0
0,Iron Maiden,140.0
1,U2,107.0
2,Metallica,91.0
3,Led Zeppelin,87.0
4,Os Paralamas Do Sucesso,45.0
5,Deep Purple,44.0
6,Faith No More,42.0
7,Lost,41.0
8,Eric Clapton,40.0
9,R.E.M.,39.0


In [103]:
#Top 10 Genres
toptengenres = pd.crosstab(index = songs["Genre"],
           columns = 'frequency',
           values = songs.Quantity,
           aggfunc = 'sum').sort_values('frequency', ascending = False).head(10)

In [104]:
top10genres = toptengenres.reset_index(level='Genre')

In [105]:
top10genres

col_0,Genre,frequency
0,Rock,835.0
1,Latin,386.0
2,Metal,264.0
3,Alternative & Punk,244.0
4,Jazz,80.0
5,Blues,61.0
6,TV Shows,47.0
7,Classical,41.0
8,R&B/Soul,41.0
9,Reggae,30.0


In [106]:
songs['Year'] = songs.InvoiceDate.dt.year

In [107]:
catalog = songs.loc[:, ('Song', 'Genre', 'Album', 'ArtistName', 'MediaTypes', 'UnitPrice')]

In [115]:
songs.dtypes

Song                    object
Genre                 category
Album                 category
ArtistName            category
MediaTypes            category
UnitPrice              float64
Composer                object
CustomerId             float64
customername            object
Quantity               float64
InvoiceDate     datetime64[ns]
City                  category
Country               category
TrackId                  int64
Year                     int64
dtype: object

In [108]:
sales = songs.copy()

In [109]:
sales

,Song,Genre,Album,ArtistName,MediaTypes,UnitPrice,Composer,CustomerId,customername,Quantity,InvoiceDate,City,Country,TrackId,Year
0,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,MPEG audio file,0.99,"Angus Young, Malcolm Young, Brian Johnson",47.0,Lucas Mancini,1.0,2010-04-13,Rome,Italy,1,2010
1,Balls to the Wall,Rock,Balls to the Wall,Accept,Protected AAC audio file,0.99,Unknown,2.0,Leonie Köhler,1.0,2009-01-01,Stuttgart,Germany,2,2009
2,Balls to the Wall,Rock,Balls to the Wall,Accept,Protected AAC audio file,0.99,Unknown,33.0,Ellie Sullivan,1.0,2011-07-25,Yellowknife,Canada,2,2011
3,Fast As a Shark,Rock,Restless and Wild,Accept,Protected AAC audio file,0.99,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",13.0,Fernanda Ramos,1.0,2012-11-01,Brasília,Brazil,3,2012
4,Restless and Wild,Rock,Restless and Wild,Accept,Protected AAC audio file,0.99,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",2.0,Leonie Köhler,1.0,2009-01-01,Stuttgart,Germany,4,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,"Symphony No. 2, Op. 16 - ""The Four Temperamen...",Classical,Nielsen: The Six Symphonies,Göteborgs Symfoniker & Neeme Järvi,Protected AAC audio file,0.99,Carl Nielsen,13.0,Fernanda Ramos,1.0,2012-11-01,Brasília,Brazil,3494,2012
2236,"Étude 1, In C Major - Preludio (Presto) - Liszt",Classical,Liszt - 12 Études D'Execution Transcendante,Michele Campanella,Purchased AAC audio file,0.99,Unknown,47.0,Lucas Mancini,1.0,2010-04-13,Rome,Italy,3496,2010
2237,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Classical,Respighi:Pines of Rome,Eugene Ormandy,Protected AAC audio file,0.99,Unknown,33.0,Ellie Sullivan,1.0,2011-07-25,Yellowknife,Canada,3499,2011
2238,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Protected AAC audio file,0.99,Franz Schubert,47.0,Lucas Mancini,1.0,2010-04-13,Rome,Italy,3500,2010


In [110]:
sales.Year = sales.Year.apply(str)

In [113]:
sales.Year = sales.Year.apply(str)
sales.UnitPrice = sales.UnitPrice.apply(str)
sales.Quantity = sales.Quantity.apply(str)
sales.CustomerId = sales.CustomerId.apply(str)
sales.TrackId = sales.TrackId.apply(str)

In [114]:
sales.dtypes

Song                    object
Genre                 category
Album                 category
ArtistName            category
MediaTypes            category
UnitPrice               object
Composer                object
CustomerId              object
customername            object
Quantity                object
InvoiceDate     datetime64[ns]
City                  category
Country               category
TrackId                 object
Year                    object
dtype: object

In [111]:
sales.dtypes

Song                    object
Genre                 category
Album                 category
ArtistName            category
MediaTypes            category
UnitPrice              float64
Composer                object
CustomerId             float64
customername            object
Quantity               float64
InvoiceDate     datetime64[ns]
City                  category
Country               category
TrackId                  int64
Year                    object
dtype: object

In [79]:
sales.Year = sales.Year.astype('category')